In [57]:
import pandas as pd
import numpy as np
import nltk
import pickle
from string import punctuation
from collections import Counter
import itertools as it
from sklearn.preprocessing import LabelEncoder

PATH_DATA = '../data/flickr/Flickr8k_text/Flickr8k.lemma.token.txt'

stop = nltk.corpus.stopwords.words('english')

In [53]:
df = pd.read_csv(PATH_DATA, sep='\t', header=None)
df = pd.concat([pd.DataFrame(df[0].str.split('#').tolist()), df[1]], axis=1)
df.columns = ['img', 'cap#', 'cap']

In [54]:
df['img'] = df['img'].astype('category')

In [55]:
df.to_msgpack('../data/flickr/Flickr8k_text/df.msg')

In [23]:
%%time
purge_words = stop + list(punctuation) + ['"', '\'', '\\']
data_words = df.cap\
    .apply(lambda row: [word.lower()
                        for word in nltk.word_tokenize(row) 
                        if word.lower() not in purge_words])

CPU times: user 4.88 s, sys: 0 ns, total: 4.88 s
Wall time: 4.89 s


In [35]:
vocab = set(it.chain(*data_words))
vocab_size = len(vocab) + 1  # +1 for UNK

max_len = max(map(len, data_words))

In [44]:
%%time
le = LabelEncoder()
le.fit(list(vocab))
le.classes_ = np.insert(le.classes_, 0, '')  # UNK tok @ 0

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 6.54 ms


In [45]:
enc_d = dict(zip(le.classes_, range(len(le.classes_))))

In [46]:
%%time
data_words_enc = data_words.map(lambda l: list(map(enc_d.get, l)))

CPU times: user 108 ms, sys: 4 ms, total: 112 ms
Wall time: 109 ms


In [47]:
data_words_enc.to_msgpack('../data/flickr/Flickr8k_text/txt_enc_s.msg', compress='blosc')

In [50]:
pickle.dump(list(le.classes_), open('../data/flickr/Flickr8k_text/vocab.p', 'wb'))